In [1]:
import os
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torchvision
from google.colab import drive
from tqdm.notebook import tqdm

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/MyDrive/FinalProjectTraining/Data'

In [4]:
zip_path = os.path.join(data_path, 'img_align_celeba.zip')
attr_path = os.path.join(data_path, 'list_attr_celeba.csv')
partition_path = os.path.join(data_path, 'list_eval_partition.csv')
bbox_path = os.path.join(data_path, 'list_bbox_celeba.csv')
landmarks_path = os.path.join(data_path, 'list_landmarks_align_celeba.csv')

extract Attributes

In [7]:
df_attr=pd.read_csv(attr_path)

In [8]:
df_attr.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [13]:
# Filter only the desired attributes
target_attributes = ['Smiling', 'Male', 'Wearing_Hat']
df_attr_filtered = df_attr[['image_id'] + target_attributes]

In [14]:
df_attr_filtered.head()

,image_id,Smiling,Male,Wearing_Hat
0,000001.jpg,1,-1,-1
1,000002.jpg,1,-1,-1
2,000003.jpg,-1,1,-1
3,000004.jpg,-1,-1,-1
4,000005.jpg,-1,-1,-1


In [15]:
# Convert all {-1, 1} to {0, 1}
attr_columns = ['Smiling', 'Male', 'Wearing_Hat']
df_attr_filtered[attr_columns] = df_attr_filtered[attr_columns].applymap(lambda x: 1 if x == 1 else 0)

<ipython-input-15-d580c8b675fa>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_attr_filtered[attr_columns] = df_attr_filtered[attr_columns].applymap(lambda x: 1 if x == 1 else 0)
<ipython-input-15-d580c8b675fa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr_filtered[attr_columns] = df_attr_filtered[attr_columns].applymap(lambda x: 1 if x == 1 else 0)


In [16]:
df_attr_filtered.head()

,image_id,Smiling,Male,Wearing_Hat
0,000001.jpg,1,0,0
1,000002.jpg,1,0,0
2,000003.jpg,0,1,0
3,000004.jpg,0,0,0
4,000005.jpg,0,0,0


0 = training

1 = validation

2 = testing

In [5]:
df_partition = pd.read_csv(partition_path)

In [6]:
df_partition.head()

,image_id,partition
0,000001.jpg,0
1,000002.jpg,0
2,000003.jpg,0
3,000004.jpg,0
4,000005.jpg,0


In [17]:
df_train = df_partition[df_partition['partition'] == 0]
df_val = df_partition[df_partition['partition'] == 1]
df_test = df_partition[df_partition['partition'] == 2]

In [20]:
print(df_train.shape)
print(df_test.shape)
print(df_val.shape)

(162770, 2)
(19962, 2)
(19867, 2)


In [21]:
# Merge with attribute labels
df_train = pd.merge(df_train, df_attr_filtered, on='image_id')
df_val = pd.merge(df_val, df_attr_filtered, on='image_id')
df_test = pd.merge(df_test, df_attr_filtered, on='image_id')

In [22]:
df_train.head()

,image_id,partition,Smiling,Male,Wearing_Hat
0,000001.jpg,0,1,0,0
1,000002.jpg,0,1,0,0
2,000003.jpg,0,0,1,0
3,000004.jpg,0,0,0,0
4,000005.jpg,0,0,0,0


In [23]:
df_train.drop(columns=['partition'], inplace=True)
df_val.drop(columns=['partition'], inplace=True)
df_test.drop(columns=['partition'], inplace=True)

In [24]:
SAVE_PATH = '/content/drive/MyDrive/FinalProjectTraining/Data'
df_train.to_csv(os.path.join(SAVE_PATH, 'celeba_train_labels.csv'), index=False)
df_val.to_csv(os.path.join(SAVE_PATH, 'celeba_val_labels.csv'), index=False)
df_test.to_csv(os.path.join(SAVE_PATH, 'celeba_test_labels.csv'), index=False)